In [8]:
import numpy as np
from DHTVPac.MyPloter import plot_with_gradient_map
from DHTVPac.HTV_Delaunay import MyDelaunay
from DHTVPac.opt_solver import double_fista
import pickle

In [9]:
data = np.loadtxt('data/cervin_gt.txt', skiprows=0)
X = data[:, :2]
y = data[:, 2]

tri_gt = MyDelaunay(X, y)

np.random.seed(42)
train_points = np.random.uniform(np.min(X), np.max(X), size=(4800, 2))
train_values = tri_gt.evaluate(train_points)

load_model = True

if not load_model:
    tri = MyDelaunay(train_points, train_values)
    tri.construct_forward_matrix()
    tri.construct_regularization_matrix()
    file_to_store = open("cervin_results/cervin.pickle", "wb")
    pickle.dump(tri, file_to_store)
    file_to_store.close()
else:
    file_to_read = open("cervin_results/cervin.pickle", "rb")
    tri = pickle.load(file_to_read)
    file_to_read.close()

lmbda = 0.01
load_vals = True

if not load_vals:
    y_fista, _ = double_fista(tri.data_values, tri.H, tri.L, tri.lip_H, tri.lip_L, lmbda, 1, 100000, device='cuda:0', verbose=False)
    fista_solution = y_fista.detach().cpu().numpy()
    np.savetxt('cervin_results/fista_sol_' + str(lmbda) + '.txt', fista_solution)
else:
    fista_solution = np.loadtxt('cervin_results/fista_sol_' + str(lmbda) + '.txt')


tri.update_values(fista_solution)

grads = tri.lat_coeffs
grad_norms = np.linalg.norm(grads, 2, axis=1)
lip = np.max(grad_norms)
print('lambda is ', lmbda, 'Lip is ', np.round(lip, 2))

plot_with_gradient_map(tri, 0, 0, 1.2, lmbda, save_plot=True, ref="cervin_pics/cervin" + str(lmbda) + "_up.pdf")
plot_with_gradient_map(tri, 1.15, 1.15, 0.4, lmbda, save_plot=True, ref="cervin_pics/cervin" + str(lmbda) + "_side.pdf")

lambda is  0.01 Lip is  4.95
